In [1]:
'''
Author: Ngawang Gurung
Date: 2024/07/24
'''

'\nAuthor: Ngawang Gurung\nDate: 2024/07/24\n'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import random, time

In [3]:
spark = SparkSession.builder \
    .appName("Spark Streaming") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/home/user/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/user/.ivy2/cache
The jars for the packages stored in: /home/user/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-63dcd820-f059-4de4-88e7-fe1ac54788ec;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-

In [4]:
KAFKA_TOPIC = "fc_txn_base"
# KAFKA_TOPIC = "am"
KAFKA_BOOTSTRAP_SERVERS = "localhost:9092"

In [5]:
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS) \
    .option("subscribe", KAFKA_TOPIC) \
    .option("startingOffsets", "earliest") \
    .load()

In [6]:
df1 = df.selectExpr("CAST(value AS STRING)", "timestamp")
schema_string = "tran_date DATE, account_number STRING, branch STRING, \
                product STRING, lcy_amount DOUBLE, transaction_code STRING, \
                description1 STRING, dc_indicator STRING, is_salary INT"

# schema_string = "account_number STRING, customer_code STRING, product STRING, \
#                 product_category STRING, acc_open_date DATETIME, acc_closed_date DATETIME, active_flag INT"
            
df2 = df1.select(from_csv(col("value"), schema_string).alias("mytable"), "timestamp")
df3 = df2.select("mytable.*", "timestamp")
df3.createOrReplaceTempView("table_view");
df4 = spark.sql("SELECT * FROM table_view")

In [7]:
my_write_stream = df4.writeStream \
        .trigger(processingTime='5 seconds') \
        .outputMode("append") \
        .option("truncate", "false") \
        .format("memory") \
        .queryName("testedTable") \
        .start()
my_write_stream.awaitTermination(1)

False

In [9]:
df = spark.sql("SELECT * FROM testedTable")
df.show()

+----------+-------------------+------+-------+----------+----------------+--------------------+------------+---------+--------------------+
| tran_date|     account_number|branch|product|lcy_amount|transaction_code|        description1|dc_indicator|is_salary|           timestamp|
+----------+-------------------+------+-------+----------+----------------+--------------------+------------+---------+--------------------+
|2020-06-09|02XYZXYZ10017529992|    15|    SBA|   14500.0|              CI|ATM WDL /00463701...|    withdraw|        0|2024-07-24 16:59:...|
|2020-06-14|02XYZXYZ10017529992|    15|    SBA|   20000.0|              CI|ATM WDL /00463701...|    withdraw|        0|2024-07-24 16:59:...|
|2020-06-01|02XYZXYZ10017517823|    15|    SBA|    4000.0|              CI|ATM WDL /NICAV701...|    withdraw|        0|2024-07-24 16:59:...|
|2020-06-05|02XYZXYZ10017517823|    15|    SBA|     500.0|              CI|ATM WDL /NICAV701...|    withdraw|        0|2024-07-24 16:59:...|
|2020-06-12|0

In [10]:
df_count = df.count()
df_count

129

In [12]:
df_count = df.count()
df_count

133